# Molecular Structure Checking using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of **checking** a **molecular structure**. The workflow uses the **BioExcel Building Blocks library (biobb)**. The particular structure used is the complex formed by the **SARS-CoV-2 Receptor Binding Domain and the human Angiotensin Converting Enzyme 2** (PDB code [6M0J](https://www.rcsb.org/structure/6m0j)). 

**Structure checking** is a key step before setting up a protein system for **simulations**. A number of **common issues** found in structures at **Protein Data Bank** may compromise the success of the simulation, or may suggest that longer equilibration procedures are necessary.

The **workflow** shows how to:

- Run **basic manipulations on structures** (selection of models, chains, alternative locations
- Detect and fix **amide assignments** and **wrong chiralities**
- Detect and fix **protein backbone** issues (missing fragments, and atoms, capping)
- Detect and fix **missing side-chain atoms**
- **Add hydrogen atoms** according to several criteria
- Detect and classify **atomic clashes**
- Detect possible **disulfide bonds (SS)**

An implementation of this workflow in a **web-based Graphical User Interface (GUI)** can be found in the https://mmb.irbbarcelona.org/biobb-wfs/ server (see https://mmb.irbbarcelona.org/biobb-wfs/help/create/structure#check).

***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to check and model 3D structures, create mutations or reconstruct missing atoms.
 - [biobb_amber](https://github.com/bioexcel/biobb_amber): Tools to setup and simulate atomistic MD simulations using AMBER MD package.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to perform chemoinformatics on molecular structures.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_structure_checking.git
cd biobb_wf_structure_checking
conda env create -f conda_env/environment.yml
conda activate biobb_wf_structure_checking
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_cmip/notebooks/biobb_wf_structure_checking.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB structure](#fetch)
 3. [PDB structure checking](#PDBchecking)
     1. [Models](#models)
     2. [Chains](#chains)
     3. [Alternative Locations](#altlocs) 
     4. [Metal Ions](#metals)
     5. [Ligands](#ligands)
     6. [Hydrogen Atoms](#hydrogens)
     7. [Water Molecules](#water)
     8. [Amide Groups](#amide)
     9. [Chirality](#chirality)
     10. [Disulfide Bridges](#ss)
     11. [Missing Side Chain Atoms](#sidechains)
     12. [Missing Backbone Atoms](#backbone)     
     13. [Atomic Clashes](#clashes)
     14. [Final Check](#finalcheck)
 8. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

In [1]:
# TO BE REMOVED!!!

%load_ext autoreload
%autoreload 2

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 6M0J)

In [1]:
import nglview
import ipywidgets

pdbCode = "6m0j" # Should be replaced to check different structures 

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [2]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode,
    'api_id' : 'mmb'
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2022-07-05 08:06:09,650 [MainThread  ] [INFO ]  Downloading: 6m0j from: http://mmb.irbbarcelona.org/api/pdb/6m0j/coords/?
2022-07-05 08:06:10,359 [MainThread  ] [INFO ]  Writting pdb to: 6m0j.pdb
2022-07-05 08:06:10,360 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**: 

In [3]:
# Show original structure
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

<a id="PDBchecking"></a>
***
## PDB structure checking
The first step of the workflow is a complete **checking** of the **molecular structure**. The **BioBB** utility **structure_check** parses the whole **PDB file**, runs a complete **structure checking**, saves **useful information** about the structure and looks for **possible issues**. The block is printing a **report** of the information extracted from the **input structure**. 
***
**Building Blocks** used:
 - [structure_check](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.structure_check) from **biobb_structure_utils.utils.structure_check**
***

In [3]:
from biobb_structure_utils.utils.structure_check import structure_check

report = pdbCode + ".report.json"

prop = {
    'features': None # This should work without properies as "None" is marked as default, but it doesn't
}

structure_check(
    input_structure_path=downloaded_pdb,
    output_summary_path=report,
    properties=prop
)

2022-07-05 08:06:15,774 [MainThread  ] [INFO ]  No features provided, all features will be computed: models, chains, altloc, metals, ligands, chiral, getss, cistransbck, backbone, amide, clashes
2022-07-05 08:06:15,775 [MainThread  ] [INFO ]  Creating e7ab8618-5a7a-47ea-8fe5-6832bca757f3 temporary folder
2022-07-05 08:06:15,777 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:06:15,779 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb --json 6m0j.report.json --check_only --non_interactive command_list --list e7ab8618-5a7a-47ea-8fe5-6832bca757f3/command_list.lst

2022-07-05 08:06:35,389 [MainThread  ] [INFO ]  Exit code 0

2022-07-05 08:06:35,391 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri,

0

<a id="StepByStep"></a>
***
## Structure checking, step by step
The **workflow** will go through the **report**, exploring all the different **features** analysed. The following cell is showing the **full content** of the **report**.
***

In [4]:
import json
with open(report, 'r') as f:
  data = json.load(f)
print(json.dumps(data, indent=2))

{
  "altloc": {
    "GLN E493": {
      "CA": [
        {
          "loc_label": "A",
          "occupancy": 0.5
        },
        {
          "loc_label": "B",
          "occupancy": 0.5
        }
      ],
      "CB": [
        {
          "loc_label": "A",
          "occupancy": 0.5
        },
        {
          "loc_label": "B",
          "occupancy": 0.5
        }
      ],
      "CD": [
        {
          "loc_label": "A",
          "occupancy": 0.5
        },
        {
          "loc_label": "B",
          "occupancy": 0.5
        }
      ],
      "CG": [
        {
          "loc_label": "A",
          "occupancy": 0.5
        },
        {
          "loc_label": "B",
          "occupancy": 0.5
        }
      ],
      "NE2": [
        {
          "loc_label": "A",
          "occupancy": 0.5
        },
        {
          "loc_label": "B",
          "occupancy": 0.5
        }
      ],
      "OE1": [
        {
          "loc_label": "A",
          "occupancy": 0.5
        },
    

<a id="models"></a>
##  Models
Presence of **models** in the structure. MD simulations require a single structure, although some structures (e.g. biounits) may be defined as a series of models, in such case all of them are usually required.
***
Use **BioBB extract_model** to select the desired **models**. 

 - [extract_model](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_model) from **biobb_structure_utils.utils.extract_model**
***

In [5]:
print(json.dumps(data['models'], indent=2))

{
  "nmodels": 1
}


In [6]:
from biobb_structure_utils.utils.extract_model import extract_model

pdb_models = pdbCode + ".models.pdb"

prop = {
    'models': [ 1 ]
}

extract_model(
    input_structure_path=downloaded_pdb,
    output_structure_path=pdb_models,
    properties=prop
)

2022-07-05 08:07:00,853 [MainThread  ] [INFO ]  Creating 40bb4969-ef9d-496c-b30b-18bf82caae5a temporary folder
2022-07-05 08:07:00,854 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:07:00,855 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb -o 40bb4969-ef9d-496c-b30b-18bf82caae5a/model1.pdb --force_save models --select 1

2022-07-05 08:07:01,702 [MainThread  ] [INFO ]  Exit code 0

2022-07-05 08:07:01,705 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. c

0

In [29]:
# Show modified structure
view = nglview.show_structure_file(pdb_models)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

<a id="chains"></a>
##  Chains
Presence of **chains** in the structure. MD simulations are usually performed with complete structures. However input structure may contain **several copies of the system**, or contain **additional chains** like **peptides** or **nucleic acids** that may be removed. 
***
Use **BioBB extract_chain** to select the desired **chains**. 

 - [extract_chain](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_chain) from **biobb_structure_utils.utils.extract_chain**
***

In [7]:
print(json.dumps(data['chains'], indent=2))

{
  "ids": {
    "A": 1,
    "E": 1
  }
}


In [8]:
from biobb_structure_utils.utils.extract_chain import extract_chain

pdb_chains = pdbCode + ".chains.pdb"

prop = {
    'chains': [ 'A', 'E' ]
}

extract_chain(
    input_structure_path=pdb_models,
    output_structure_path=pdb_chains,
    properties=prop
)

2022-07-05 08:07:10,814 [MainThread  ] [INFO ]  Selected Chains: A,E
2022-07-05 08:07:10,815 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:07:10,816 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.models.pdb -o 6m0j.chains.pdb --force_save chains --select A,E

2022-07-05 08:07:11,854 [MainThread  ] [INFO ]  Exit code 0

2022-07-05 08:07:11,856 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.models.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Nu

0

In [10]:
# Show modified structure
view = nglview.show_structure_file(pdb_chains)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

<a id="altlocs"></a>
##  Alternative Locations
Presence of residues with **alternative locations**. Atoms with **alternative coordinates** and their **occupancy** are reported. MD simulations requires a **single position** for each atom. 
***
Use **BioBB fix_altloc** to select the desired **alternative locations**. 

 - [fix_altloc](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_altloc) from **biobb_model.model.fix_altloc**
***

In [11]:
print(json.dumps(data['altloc'], indent=2))

{
  "GLN E493": {
    "CA": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "CB": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "CD": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "CG": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "NE2": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ],
    "OE1": [
      {
        "loc_label": "A",
        "occupancy": 0.5
      },
      {
        "loc_label": "B",
        "occupancy": 0.5
      }
    ]
  },
  "HIS A228": {
    "CA": [
      {
        "loc_lab

In [ ]:
from biobb_model.model.fix_altloc import fix_altloc

pdb_altloc = pdbCode + ".altloc.pdb"

prop = { 
    'restart': False 
}

fix_altloc(
    input_pdb_path=pdb_chains,
    output_pdb_path=pdb_altloc,
    properties=prop
)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_altloc)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="metals"></a>
##  Metal Ions
Presence of heteroatoms being **metal ions**. Only structural **metal ions** should be kept in MD simulations, as they require special **force field parameters**. In some cases, the **atomic coordination** between the **protein atoms** and the **metal ions** should be also specified. 
***
Use **BioBB remove_ligand** to remove the undesired **metal ions**. 

 - [remove_ligand](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_ligand) from **biobb_structure-utils.utils.remove_ligand**
***

In [35]:
print(json.dumps(data['metals'], indent=2))

{}


In [ ]:
from biobb_structure_utils.utils.remove_ligand import remove_ligand

pdb_metals = pdbCode + ".metals.pdb"

prop = {
    'ligand': 'ZN2'
}

remove_ligand(
    input_structure_path=pdb_altloc,
    output_structure_path=pdb_metals,
    properties=prop
)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_metals)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="ligands"></a>
##  Ligands
Presence of heteroatoms being **ligands**. Only important **ligands** should be kept in MD simulations, as they require special **force field parameters**. 
***
Use **BioBB remove_ligand** to remove the undesired **ligands**. 

 - [remove_ligand](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_ligand) from **biobb_structure-utils.utils.remove_ligand**
***

In [36]:
print(json.dumps(data['ligands'], indent=2))

{}


In [ ]:
from biobb_structure_utils.utils.remove_ligand import remove_ligand

pdb_ligands = pdbCode + ".ligands.pdb"

prop = {
    'ligand': 'NAG'
}

remove_ligand(
    input_structure_path=pdb_metals,
    output_structure_path=pdb_ligands,
    properties=prop
)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_ligands)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="hydrogens"></a>
##  Hydrogen atoms
Presence of **hydrogen atoms**. MD setup can be done with the original **hydrogen atoms**, however to prevent possible problems coming from non **standard labelling**, removing them is safer. 
***
Use **BioBB reduce_remove_hydrogens** to remove the undesired **hydrogen atoms**. 

 - [reduce_remove_hydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_remove_hydrogens) from **biobb_chemistry.ambertools.reduce_remove_hydrogens**
***

In [39]:
# No hi ha res que marqui hidrògens a l'estructura al report json (crec)
#print(json.dumps(data['hydrogens'], indent=2))

In [9]:
from biobb_chemistry.ambertools.reduce_remove_hydrogens import reduce_remove_hydrogens

pdb_ligands = pdb_chains # TO BE REMOVED!!!

pdb_hydrogens = pdbCode + ".hydrogens.pdb"

reduce_remove_hydrogens(
    input_path=pdb_ligands,
    output_path=pdb_hydrogens
)

2022-07-04 13:16:43,615 [MainThread  ] [INFO ]  Not using any container
2022-07-04 13:16:43,616 [MainThread  ] [INFO ]  reduce -Trim /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.chains.pdb > 6m0j.hydrogens.pdb

2022-07-04 13:16:43,739 [MainThread  ] [INFO ]  Exit code 0

2022-07-04 13:16:43,743 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "/Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.chains.pdb"
Trimming: removed 0 hydrogens (0 hets)
If you publish work which uses reduce, please cite:
Word, et. al. (1999) J. Mol. Biol. 285, 1735-1747.
For more information see http://kinemage.biochem.duke.edu



0

In [10]:
# Show modified structure
view = nglview.show_structure_file(pdb_hydrogens)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

<a id="water"></a>
##  Water molecules
Presence of **water molecules**. **Crystallographic water molecules** may be relevant for keeping the structure stable, however in most cases only some of them are required. These can be later added using other methods.
***
Use **BioBB remove_pdb_water** to remove the undesired **water molecules**. 

 - [remove_pdb_water](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_pdb_water) from **biobb_structure-utils.utils.remove_pdb_water**
***

In [14]:
print(json.dumps(data['stats']['num_wat'], indent=2))

0


In [15]:
from biobb_structure_utils.utils.remove_pdb_water import remove_pdb_water

pdb_water = pdbCode + ".water.pdb"

remove_pdb_water(
    input_pdb_path=pdb_hydrogens,
    output_pdb_path=pdb_water
)

2022-07-04 13:24:17,016 [MainThread  ] [INFO ]  Not using any container
2022-07-04 13:24:17,017 [MainThread  ] [INFO ]  check_structure -i 6m0j.hydrogens.pdb -o 6m0j.water.pdb --force_save water --remove yes

2022-07-04 13:24:18,028 [MainThread  ] [INFO ]  Exit code 0

2022-07-04 13:24:18,032 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 6m0j.hydrogens.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 2 (A: Protein, E: Protein)
 Num. residues:  791
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  6406

Running water. Options: --remove yes
No water molecules found
Structure not modified, saving due to --force_save option
Fin

0

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_water)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="amide"></a>
##  Amide groups
Presence of incorrect **amide groups**. **Amide terminal atoms** in ***Asparagines*** and ***Glutamines*** residues can be labelled incorrectly, with swapped **Nitrogen** and **Oxygen** atoms from the **amide group**. 
***
Use **BioBB fix_amides** to fix the incorrectly assigned **amide groups**. 

 - [fix_amides](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_amides) from **biobb_model.model.fix_amides**
***

In [16]:
print(json.dumps(data['amide'], indent=2))

{
  "detected": [
    {
      "at1": "LYS A31.NZ",
      "at2": "GLN E493.NE2",
      "dist": 2.9256
    },
    {
      "at1": "GLN A42.NE2",
      "at2": "GLN E498.NE2",
      "dist": 2.9272
    },
    {
      "at1": "ASN A103.OD1",
      "at2": "ASN A194.OD1",
      "dist": 2.8068
    },
    {
      "at1": "ASN A134.OD1",
      "at2": "GLU A140.OE2",
      "dist": 2.7852
    },
    {
      "at1": "ASN A134.ND2",
      "at2": "ASN A137.N",
      "dist": 3.0824
    },
    {
      "at1": "GLU A150.O",
      "at2": "ASN A154.OD1",
      "dist": 2.8946
    },
    {
      "at1": "ARG E357.NH1",
      "at2": "ASN E394.ND2",
      "dist": 2.9629
    }
  ],
  "n_amides": 92
}


In [ ]:
from biobb_model.model.fix_amides import fix_amides

pdb_amides = pdbCode + ".amides.pdb"

fix_amides(
    input_pdb_path=pdb_water,
    output_pdb_path=pdb_amides
)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_amides)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="chirality"></a>
##  Chirality
Presence of incorrect **chiralities**. Side chains of ***Threonine*** and ***Isoleucine*** residues are **chiral**. <br> **Incorrect atom labelling** can lead to the **wrong chirality**. 
***
Use **BioBB fix_chirality** to fix the incorrectly assigned **chirality**. 

 - [fix_chirality](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_chirality) from **biobb_model.model.fix_chirality**
***

In [18]:
print(json.dumps(data['chiral'], indent=2))

{
  "n_chirals": 72
}


In [ ]:
from biobb_model.model.fix_chirality import fix_chirality

pdb_chiral = pdbCode + ".chiral.pdb"

fix_chirality(input_pdb_path=pdb_amides,
              output_pdb_path=pdb_chiral,
              properties=prop)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_chiral)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="ss"></a>
##  Disulfide Bridges
Presence of **disulfide bridges** (-S-S- bonds) based on distance criteria. **Disulfide bonds** are crucial for the **structure stability**, and MD simulations require those bonds to be correctly set. Most of the **MD packages** come with tools able to automatically identify -S-S- bonds and include them in the **system topology** parameters, but some of them need the residues involved in the bond to be **explicitly marked** (e.g. AMBER CYX residues). 
***
Use **BioBB fix_ss_bonds** to mark the **disulfide bridges**. 

 - [fix_ss_bonds](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_ss_bonds) from **biobb_model.model.fix_ss_bonds**
***

In [22]:
print(json.dumps(data['getss'], indent=2))

{
  "found": [
    {
      "at1": "CYS A133.SG",
      "at2": "CYS A141.SG",
      "dist": 4.237
    },
    {
      "at1": "CYS A344.SG",
      "at2": "CYS A361.SG",
      "dist": 4.1587
    },
    {
      "at1": "CYS A530.SG",
      "at2": "CYS A542.SG",
      "dist": 4.0947
    },
    {
      "at1": "CYS E336.SG",
      "at2": "CYS E361.SG",
      "dist": 4.1517
    },
    {
      "at1": "CYS E379.SG",
      "at2": "CYS E432.SG",
      "dist": 4.177
    },
    {
      "at1": "CYS E391.SG",
      "at2": "CYS E525.SG",
      "dist": 4.1912
    },
    {
      "at1": "CYS E480.SG",
      "at2": "CYS E488.SG",
      "dist": 4.2686
    }
  ]
}


In [ ]:
from biobb_model.model.fix_ssbonds import fix_ssbonds

pdb_ssbonds = pdbCode + ".ssbonds.pdb"

fix_ssbonds(input_pdb_path=pdb_chiral,
              output_pdb_path=pdb_ssbonds,
              properties=prop)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_ssbonds)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="sidechains"></a>
##  Side Chains
Presence of **missing protein side chain atoms**. MD programs only work with complete residues; some of the MD packages come with tools able to **model missing side chain atoms**, but not all of them. 
***
Use **BioBB fix_side_chain** to model the **missing side chain atoms**. 

 - [fix_side_chain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**
***

In [ ]:
# No hi ha res que marqui missing side chain atoms al report json (crec)
#print(json.dumps(data['side_chain'], indent=2))

In [ ]:
from biobb_model.model.fix_side_chain import fix_side_chain

pdb_side_chains = pdbCode + ".sidechains.pdb"

prop = { 
    'use_modeller': True 
}

fix_side_chain(
    input_pdb_path=pdb_ssbonds,
    output_pdb_path=pdb_side_chains,
    properties=prop
)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_side_chains)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="backbone"></a>
##  Backbone
Presence of **missing backbone atoms**. **PDB files** can have **missing backbone atoms**, usually placed in **flexible regions** of the molecule. **MD programs** work with amino acid libraries that need all atoms to be present. Some MD packages tools are able to model **side chain missing atoms**, but they are rarely capable of modeling **backbone atoms**. 
***
Use **BioBB fix_backbone** to model the **missing backbone atoms**. <br>
***NOTE:*** *This building block is using the Modeler tool, which requires an academic license.*

 - [fix_backbone](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_backbone) from **biobb_model.model.fix_backbone**
 
An additional **building block** is used to extract the **canonical FASTA sequence** for the protein, to be used in **Modeler** to model the **missing backbone region**:

 - [canonical_fasta](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.canonical_fasta) from **biobb_io.api.canonical_fasta**

***

In [25]:
print(json.dumps(data['backbone'], indent=2))

{
  "missing_atoms": {
    "ASP A615": [
      "OXT"
    ],
    "GLY E526": [
      "OXT"
    ]
  }
}


In [23]:
from biobb_io.api.canonical_fasta import canonical_fasta

pdb_fasta = pdbCode + ".fasta"

prop = {
    'pdb_code': pdbCode,
    'api_id': 'pdb'
}

canonical_fasta(
    output_fasta_path=pdb_fasta,
    properties=prop
)

2022-07-04 16:34:43,400 [MainThread  ] [INFO ]  Downloading: 6m0j from: https://www.rcsb.org/fasta/entry/6m0j
2022-07-04 16:34:44,320 [MainThread  ] [INFO ]  Writting FASTA to: 6m0j.fasta


/anaconda3/envs/biobb_wf_structure_checking_model/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.rcsb.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0

In [ ]:
from biobb_model.model.fix_backbone import fix_backbone

pdb_backbone = pdbCode + ".backbone.pdb"

fix_backbone(
    input_pdb_path=pdb_side_chains,
    input_fasta_canonical_sequence_path=pdb_fasta,
    output_pdb_path=pdb_backbone,
    properties=prop
)

In [ ]:
# Show modified structure
view = nglview.show_structure_file(pdb_backbone)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

<a id="clashes"></a>
##  Atomic Clashes
Presence of **atomic clashes**. Atoms that are **too close in space** can have a problem of **energetic repulsion**. Most clashes come from **over-compactation** of crystal structures and are naturally corrected on **system setup** or **MD equilibration**, but may lead to a significant **distortion** of the structure. **Clashes** are detected based on **distance criteria** and are classified in different groups, depending on the **atom types** involved:

- **Severe**: Atoms too close, usually indicating superimposed structures or badly modelled regions. Should be fixed.
- **Apolar**: Vdw collisions. Usually fixed during the simulation.
- **Polar and ionic**: Usually indicate wrong side chain conformations. Usually fixed during the simulation

***
Use the **BioBB_amber** module to energetically minimize the structure and fix **severe atomic clashes**.

 - [leap_gen_top](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_gen_top) from **biobb_amber.leap.leap_gen_top**
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander.sander_mdrun**
 - [amber_to_pdb](https://biobb-amber.readthedocs.io/en/latest/ambpdb.html#module-ambpdb.amber_to_pdb) from **biobb_amber.ambpdb.amber_to_pdb**

***

In [ ]:
print(json.dumps(data['clashes'], indent=2))

In [9]:
from biobb_amber.leap.leap_gen_top import leap_gen_top

pdb_amber = pdbCode + ".amber.pdb"
top_amber = pdbCode + ".amber.top"
crd_amber = pdbCode + ".amber.crd"

prop = {
    'forcefield': ['protein.ff14SB']
}

leap_gen_top(
    input_pdb_path=pdb_chains,
    output_pdb_path=pdb_amber,
    output_top_path=top_amber,
    output_crd_path=crd_amber,
    properties=prop
)

2022-07-05 08:08:59,798 [MainThread  ] [INFO ]  Creating 1f9b89b2-72a6-4615-bf84-87b096c3efe8 temporary folder
2022-07-05 08:08:59,800 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:08:59,802 [MainThread  ] [INFO ]  tleap  -f 1f9b89b2-72a6-4615-bf84-87b096c3efe8/leap.in

2022-07-05 08:09:00,907 [MainThread  ] [INFO ]  Exit code 0

2022-07-05 08:09:00,909 [MainThread  ] [INFO ]  -I: Adding /anaconda3/envs/biobb_wf_structure_checking_model/dat/leap/prep to search path.
-I: Adding /anaconda3/envs/biobb_wf_structure_checking_model/dat/leap/lib to search path.
-I: Adding /anaconda3/envs/biobb_wf_structure_checking_model/dat/leap/parm to search path.
-I: Adding /anaconda3/envs/biobb_wf_structure_checking_model/dat/leap/cmd to search path.
-f: Source 1f9b89b2-72a6-4615-bf84-87b096c3efe8/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./1f9b89b2-72a6-4615-bf84-87b096c3efe8/leap.in
----- Source: /anaconda3/envs/biobb_wf_structure_checking_model/dat/leap/cmd/leaprc

0

In [15]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

trj_amber = pdbCode + ".amber.crd"
rst_amber = pdbCode + ".amber.rst"
log_amber = pdbCode + ".amber.log"

prop = {
    'simulation_type' : 'minimization',
    'mdin' : {
        'ntb' : 0,        # Periodic Boundary. No periodicity is applied and PME (Particle Mesh Ewald) is off.
        'cut' : 12,       # Nonbonded cutoff, in Angstroms.
        'maxcyc' : 500,   # Maximum number of cycles of minimization.
        'ncyc' : 50       # Minimization will be switched from steepest descent to conjugate gradient after ncyc cycles.
    }
}

sander_mdrun(
     input_top_path=top_amber,
     input_crd_path=crd_amber,
     output_traj_path=trj_amber,
     output_rst_path=rst_amber,
     output_log_path=log_amber,
     properties=prop
)

2022-07-05 08:18:43,097 [MainThread  ] [INFO ]  Creating 10989853-4ede-4e05-8061-df26de31df8c temporary folder
2022-07-05 08:18:43,099 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:18:43,101 [MainThread  ] [INFO ]  sander -O -i 10989853-4ede-4e05-8061-df26de31df8c/sander.mdin -p 6m0j.amber.top -c 6m0j.amber.crd -r 6m0j.amber.rst -o 6m0j.amber.log -x 6m0j.amber.crd

2022-07-05 08:20:30,984 [MainThread  ] [INFO ]  Exit code 0

2022-07-05 08:20:30,987 [MainThread  ] [INFO ]  Removed: ['10989853-4ede-4e05-8061-df26de31df8c', 'mdinfo']


0

In [32]:
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb

pdb_amber_min = pdbCode + ".amber-min.pdb"

amber_to_pdb(
      input_top_path=top_amber,
      input_crd_path=rst_amber,
      output_pdb_path=pdb_amber_min
)

2022-07-05 08:41:57,615 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:41:57,616 [MainThread  ] [INFO ]  ambpdb  -p 6m0j.amber.top -c 6m0j.amber.rst >  6m0j.amber-min.pdb

2022-07-05 08:41:57,846 [MainThread  ] [INFO ]  Exit code 0



0

<a id="finalcheck"></a>
##  Final Check
Checking the **final structure** to analyse the **possible issues** present in the **modified structure** in comparison with the issues we found with the **original structure** at the beginning of the workflow. 

In [33]:
from biobb_structure_utils.utils.structure_check import structure_check

report_final = pdbCode + ".report_final.json"

prop = {
    #'features': ['clashes'] 
    'features': None  
}

structure_check(
    input_structure_path=pdb_amber_min,
    output_summary_path=report_final,
    properties=prop
)

2022-07-05 08:42:01,049 [MainThread  ] [INFO ]  No features provided, all features will be computed: models, chains, altloc, metals, ligands, chiral, getss, cistransbck, backbone, amide, clashes
2022-07-05 08:42:01,051 [MainThread  ] [INFO ]  Creating a8e21aa4-7383-4dac-9dc0-3727cfcd00ae temporary folder
2022-07-05 08:42:01,055 [MainThread  ] [INFO ]  Not using any container
2022-07-05 08:42:01,056 [MainThread  ] [INFO ]  check_structure -i /Users/hospital/biobb_tutorials/biobb_wf_structure_checking/biobb_wf_structure_checking/notebooks/6m0j.amber-min.pdb --json 6m0j.report_final.json --check_only --non_interactive command_list --list a8e21aa4-7383-4dac-9dc0-3727cfcd00ae/command_list.lst

2022-07-05 08:42:21,121 [MainThread  ] [INFO ]  Exit code 0

2022-07-05 08:42:21,124 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospi

0

In [34]:
import json
with open(report_final, 'r') as f:
  data_final = json.load(f)
print(json.dumps(data_final, indent=2))

{
  "altloc": {},
  "amide": {
    "detected": [
      {
        "at1": "ASN  85.OD1",
        "at2": "ASN  176.OD1",
        "dist": 3.023
      },
      {
        "at1": "GLY  761.O",
        "at2": "GLN  763.OE1",
        "dist": 3.056
      }
    ],
    "n_amides": 92
  },
  "backbone": {
    "breaks": {
      "detected": [
        [
          "ASP  597",
          "THR  598"
        ]
      ]
    },
    "long_links": [
      [
        "ASP  597",
        "THR  598",
        109.45083
      ]
    ]
  },
  "chains": {
    "ids": {
      " ": 1
    }
  },
  "chiral": {
    "n_chirals": 72
  },
  "cistransbck": {
    "cis": [
      [
        "GLU  127",
        "PRO  128",
        -15.758
      ]
    ],
    "unusual_trans": [
      [
        "LEU  124",
        "LEU  125",
        155.897
      ],
      [
        "PRO  128",
        "GLY  129",
        -154.762
      ],
      [
        "CYS  343",
        "THR  344",
        -156.964
      ],
      [
        "ASN  490",
        "ASP  

In [38]:
# import jsondiff
# res = jsondiff.diff(data, data_final)
# #res = jsondiff.diff(data, data_final, syntax='explicit')
# if res:
#     print("Diff found")
#     print(res)
#     #print(json.dumps(res, indent=2))
# else:
#     print("Same")

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)